In [ ]:
import os
import re
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import ptitprince as pt
from scipy.io import loadmat
import matplotlib.pyplot as plt


def plot(overlaps: np.ndarray):
    data = {
        r'$|v_1\rangle$': overlaps[:, 0],
        r'$|v_2\rangle$': overlaps[:, 1],
        r'$|v_3\rangle$': overlaps[:, 2],
        r'$|v_4\rangle$': overlaps[:, 3],
    }
    colors = ['#74B816', '#339AF0', '#9C36B5', '#C92A2A']
    df = pd.DataFrame(data)
    pt.half_violinplot(data=df, scale='count', width=0.6, \
        inner=None, linewidth=1.2, palette=colors)
    sns.stripplot(data=df, jitter=True, palette=colors, size=1)
    overlaps_max = np.ceil(overlaps.max() * 10) / 10
    plt.yticks(np.linspace(0, overlaps_max, 11))
    plt.ylabel('Overleaps')
    plt.show()


date = '202411'
path_wsl = r'\\wsl.localhost\Fedora\home\ghost\QuditVGON\mats'
path = './mats' if 'linux' in sys.platform else path_wsl
for name in sorted(os.listdir(path), reverse=True):
    match = re.search(r'(VGON_nqd\d+_\d{8}_\d{6}).mat', name)
    if match and date in name:
        load = loadmat(f'{path}/{match.group(1)}.mat')
        energy = load['energy'].item()
        n_qudits = int(re.search(r'nqd(\d+)', name).group(1))
        if 'cos_sim' in load:
            cos_sim = load['cos_sim'].item()
            info_str = f'Cos_Sim: {cos_sim:.8f}'
            if 'fidelity' in load:
                fidelity = load['fidelity'].item()
                info_str = f'Fidelity: {fidelity:.8f}, {info_str}'
        if 'cos_sim_max' in load and 'cos_sim_mean' in load:
            cos_sim_max = load['cos_sim_max'].item()
            cos_sim_mean = load['cos_sim_mean'].item()
            info_str = f'Cos_Sim: {cos_sim_max:.8f}, {cos_sim_mean:.8f}'
            if 'fidelity_max' in load and 'fidelity_mean' in load:
                fidelity_max = load['fidelity_max'].item()
                fidelity_mean = load['fidelity_mean'].item()
                info_str = f'Fidelity: {fidelity_max:.8f}, {fidelity_mean:.8f}, {info_str}'
        else:
            fidelity = load['fidelity'].item()
            info_str = f'Fidelity: {fidelity:.8f}'
        n_train = (load['n_train'] if 'n_train' in load else load['n_iter']).item()
        if 'overlaps' in load:
            if 'fidelity_max' in load.keys() and energy < -3.99:
                overlaps = load['overlaps']
                print(f'{name}, Energy: {energy:.8f}, {info_str}, {n_train}, {overlaps.shape[0]}')
                plot(overlaps)
        else:
            info_str += ', No overlaps'
            print(f'{name}, Energy: {energy:.8f}, {info_str}, {n_train}')

In [ ]:
with open('./logs/VGON_nqd7_degeneracy_202411.log') as f:
    count = 0
    lines = f.readlines()
    for line in lines:
        energy_pattern = r'Energy: (-*\d\.\d+)'
        fidelity_pattern = r'Fidelity: (\d\.\d+)'
        energy_match = re.search(energy_pattern, line)
        fidelity_match = re.search(fidelity_pattern, line)
        if energy_match and fidelity_match:
            energy = float(energy_match.group(1))
            fidelity = float(fidelity_match.group(1))
            if energy < -3.99 and fidelity < 0.98:
                count += 1
                print(count, line.strip())

```python
seaborn version: 0.11.0

.../site-packages/seaborn/axisgrid.py
677  plot_args = [v for k, v in plot_data.items()]  # zyz: replace .iteritems() with .items()

.../site-packages/seaborn/categorical.py
81  iter_data = plot_data.items()  # zyz: replace .iteritems() with .items()

.../site-packages/seaborn/cm.py
1  import matplotlib as mpl
...
1583  # mpl_cm.register_cmap(_name, _cmap)
1584  # mpl_cm.register_cmap(_name + "_r", _cmap_r)
1586  # zyz: replace matplotlib.cm.register with matplotlib.colormaps.register
1587  mpl.colormaps.register(name=_name, cmap=_cmap)
1588  mpl.colormaps.register(name=_name + "_r", cmap=_cmap_r)
```